In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive')




Tokenization

In [ ]:
import pandas as pd
import re
import html
import warnings
import os
import torch

from bs4 import BeautifulSoup, MarkupResemblesLocatorWarning
from transformers import AutoTokenizer
from sklearn.model_selection import train_test_split

# -------------------------------
# Suppress warnings
# -------------------------------
warnings.filterwarnings("ignore", category=MarkupResemblesLocatorWarning)

# -------------------------------
# Paths
# -------------------------------
DATA_PATH = "/content/drive/MyDrive/NewsSumm Dataset.xlsx"
SAVE_DIR  = "/content/drive/MyDrive/tokenized_llama_batches"

TRAIN_DIR = os.path.join(SAVE_DIR, "train")
TEST_DIR  = os.path.join(SAVE_DIR, "test")

os.makedirs(TRAIN_DIR, exist_ok=True)
os.makedirs(TEST_DIR, exist_ok=True)

# -------------------------------
# Load dataset
# -------------------------------
df = pd.read_excel(DATA_PATH)
df = df.dropna(subset=["human_summary"])
print("Rows after cleaning:", len(df))

# -------------------------------
# Resample
# -------------------------------
df = df.sample(n=15000, random_state=42)
print("Rows after resampling:", len(df))

# -------------------------------
# Clean text
# -------------------------------
def clean_text(text):
    if not isinstance(text, str):
        return ""
    text = BeautifulSoup(text, "html.parser").get_text(" ")
    text = html.unescape(text)
    text = re.sub(r"https?://\S+|www\.\S+", "", text)
    text = re.sub(r"\b[\w\.-]+@[\w\.-]+\.\w+\b", "", text)
    text = re.sub(r"\+?\d[\d\s\-]{8,}\d", "", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text

df["human_summary"] = df["human_summary"].apply(clean_text)

# -------------------------------
# Train / Test split
# -------------------------------
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

train_texts = train_df["human_summary"].astype(str).tolist()
test_texts  = test_df["human_summary"].astype(str).tolist()

print("Train rows:", len(train_texts))
print("Test rows:", len(test_texts))

# -------------------------------
# Load tokenizer (SAFE model)
# -------------------------------
MODEL_NAME = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
# If you REALLY want llama-2:
# MODEL_NAME = "meta-llama/Llama-2-7b-hf"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)

# Required for LLaMA style models
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# -------------------------------
# Tokenization settings (SAFE)
# -------------------------------
BATCH_SIZE = 2        # safe for Colab
MAX_LEN = 128

# -------------------------------
# Batch tokenization function
# -------------------------------
def tokenize_and_save(texts, out_dir, split_name):

    print(f"\nTokenizing {split_name} set...")
    batch_id = 0

    for i in range(0, len(texts), BATCH_SIZE):

        batch_texts = texts[i:i+BATCH_SIZE]

        encoded = tokenizer(
            batch_texts,
            truncation=True,
            padding="max_length",
            max_length=MAX_LEN,
            return_tensors="pt"
        )

        input_ids = encoded["input_ids"]
        attention_mask = encoded["attention_mask"]

        # ✅ correct label masking
        labels = input_ids.clone()
        labels[labels == tokenizer.pad_token_id] = -100

        batch_data = {
            "input_ids": input_ids,
            "attention_mask": attention_mask,
            "labels": labels
        }

        save_path = os.path.join(out_dir, f"batch_{batch_id}.pt")
        torch.save(batch_data, save_path)

        if batch_id % 50 == 0:
            print(f"{split_name}: saved batch {batch_id}")

        batch_id += 1

    print(f"{split_name} tokenization done. Total batches: {batch_id}")

# -------------------------------
# Run tokenization
# -------------------------------
tokenize_and_save(train_texts, TRAIN_DIR, "TRAIN")
tokenize_and_save(test_texts, TEST_DIR, "TEST")

print("\nLLaMA-style batch-wise tokenization completed successfully")


model training

In [ ]:
import os
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from bitsandbytes.optim import Adam8bit

# --------------------
# Clean GPU memory first
# --------------------
torch.cuda.empty_cache()

# --------------------
# Paths
# --------------------
TRAIN_DIR = "/content/drive/MyDrive/tokenized_llama_batches/train/"
SAVE_DIR  = "/content/drive/MyDrive/tinyllama_finetuned"
MODEL_NAME = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

# --------------------
# Device
# --------------------
device = "cuda"
print("Device:", device)

# --------------------
# Load tokenizer
# --------------------
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# --------------------
# Load model SAFELY (FP16 directly)
# --------------------
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.float16,
    device_map="auto"
)

model.gradient_checkpointing_enable()
model.train()

# --------------------
# Optimizer (8-bit)
# --------------------
optim = Adam8bit(model.parameters(), lr=2e-5)

# --------------------
# Load batch files
# --------------------
files = sorted([f for f in os.listdir(TRAIN_DIR) if f.endswith(".pt")])
files = files[:1500]   # safer limit

print("Training batches:", len(files))

total_loss = 0.0

# --------------------
# Training loop
# --------------------
for i, fname in enumerate(files, 1):

    batch = torch.load(os.path.join(TRAIN_DIR, fname))

    input_ids = batch["input_ids"].cuda()
    attention_mask = batch["attention_mask"].cuda()
    labels = batch["labels"].cuda()

    optim.zero_grad()

    outputs = model(
        input_ids=input_ids,
        attention_mask=attention_mask,
        labels=labels
    )

    loss = outputs.loss
    loss.backward()

    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
    optim.step()

    total_loss += loss.item()

    if i % 10 == 0:
        print(f"Batch {i} | Avg loss: {total_loss / i:.4f}")

    del batch, input_ids, attention_mask, labels, outputs, loss
    torch.cuda.empty_cache()

# --------------------
# Save model
# --------------------
os.makedirs(SAVE_DIR, exist_ok=True)
model.save_pretrained(SAVE_DIR)
tokenizer.save_pretrained(SAVE_DIR)

print("TinyLlama training completed safely")


In [ ]:
!pip install  rouge-score


model evaluation

In [ ]:
import os
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from rouge_score import rouge_scorer

# --------------------
# Paths
# --------------------
TEST_DIR  = "/content/drive/MyDrive/tokenized_llama_batches/test/"
MODEL_DIR = "/content/drive/MyDrive/tinyllama_finetuned"

# --------------------
# Device
# --------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

# --------------------
# Load model & tokenizer
# --------------------
tokenizer = AutoTokenizer.from_pretrained(MODEL_DIR)
tokenizer.padding_side = "left"
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    MODEL_DIR,
    torch_dtype=torch.float16 if device.type == "cuda" else torch.float32
).to(device)

model.eval()

# --------------------
# ROUGE scorer
# --------------------
scorer = rouge_scorer.RougeScorer(["rouge1", "rouge2", "rougeL"], use_stemmer=True)

r1 = r2 = rl = 0.0
r1p = r2p = rlp = 0.0
count = 0

files = sorted([f for f in os.listdir(TEST_DIR) if f.endswith(".pt")])

# Optional limit
files = files[:800]

print("Total test batches:", len(files))

# --------------------
# Evaluation loop
# --------------------
for idx, fname in enumerate(files, 1):

    if idx % 100 == 0 or idx == 1:
        print(f"Evaluating batch {idx}/{len(files)}")

    batch = torch.load(os.path.join(TEST_DIR, fname))

    input_ids = batch["input_ids"].to(device)
    attention_mask = batch["attention_mask"].to(device)
    labels = batch["labels"]

    with torch.no_grad():
        outputs = model.generate(
            input_ids=input_ids,
            attention_mask=attention_mask,
            max_new_tokens=50,
            do_sample=False
        )

    # Extract only generated tokens
    gen_only = outputs[:, input_ids.shape[1]:]

    preds = tokenizer.batch_decode(gen_only, skip_special_tokens=True)
    labels_fixed = labels.clone()
    labels_fixed[labels_fixed == -100] = tokenizer.pad_token_id
    refs = tokenizer.batch_decode(labels_fixed, skip_special_tokens=True)


    for pred, ref in zip(preds, refs):
        scores = scorer.score(ref, pred)

        r1  += scores["rouge1"].fmeasure
        r2  += scores["rouge2"].fmeasure
        rl  += scores["rougeL"].fmeasure

        r1p += scores["rouge1"].precision
        r2p += scores["rouge2"].precision
        rlp += scores["rougeL"].precision

        count += 1

    del batch, input_ids, attention_mask, outputs
    torch.cuda.empty_cache()

# --------------------
# Final results
# --------------------
print("\nROUGE RESULTS (TinyLlama / LLaMA)")
print("Samples evaluated:", count)
print("ROUGE-1 F1:", round(r1 / count, 4))
print("ROUGE-2 F1:", round(r2 / count, 4))
print("ROUGE-L F1:", round(rl / count, 4))
print("ROUGE-1 Precision:", round(r1p / count, 4))
print("ROUGE-2 Precision:", round(r2p / count, 4))
print("ROUGE-L Precision:", round(rlp / count, 4))
